In [1]:
# Have to patch sys.path until we have tighter taucmdr/anaconda integration.
# Need to move towards taucmdr as a conda package.
import os
import sys
try:
    import taucmdr
except ImportError:
    sys.path.insert(0, os.path.join(os.environ['__TAUCMDR_HOME__'], 'packages'))
finally:
    from taucmdr.model.project import Project

In [2]:
%pylab
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
import math
import numpy as np
import operator

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
num_trials = Project.selected().experiment().num_trials
trials = Project.selected().experiment().trials(xrange(0, num_trials))
trial_data = {}
for i in xrange(0, num_trials):
    trial_data[i] = trials[i].get_data()

In [4]:
#trial0_intervals = trial0_data[0][0][0].interval_data()
#trial0_intervals.plot(kind='bar')
#plt.show()

In [5]:
trial_intervals = []
for trial in xrange(0, num_trials):
    for i in xrange(0, len(trial_data[trial])):
        for j in xrange(0, len(trial_data[trial][i])):
            for k in xrange(0, len(trial_data[trial][i][j])):
                trial_intervals.append(trial_data[trial][i][j][k].interval_data())
                
expr_intervals = pd.concat(trial_intervals)

In [6]:
# levels: 0=trial, 1=node, 2=context, 3=thread, 4=region name

from taucmdr.error import InternalError

def largest_stddev(dfs,n):
    return dfs['Exclusive'].groupby(level=3).std(ddof=0).dropna().sort_values(ascending=False, axis=0)[:n]

def largest_correlation(dfs,n):
    grouped = dfs.groupby(level=4)
    return grouped['Exclusive'].agg(lambda x: np.corrcoef(x, grouped['Inclusive'].get_group('.TAU application'))[0,1]).sort_values(ascending=False, axis=0)[:n]

def largest_exclusive(df,n):
    rows = df.nlargest(n, 'Exclusive')
    return rows['Exclusive']

def hotspots(dfs, n, flag):
    if flag == 0:
        largest = largest_exclusive(dfs,n)
    elif flag == 1:
        largest = largest_stddev(dfs,n)
    elif flag == 2:
        largest = largest_correlation(dfs,n)
    else:
        print 'Invalid flag'
    y = ['exclusive time', 'standard deviation', 'correlation to total runtime']
    print 'Hotspot Analysis Summary'
    print '='*80
    print 'The code regions with largest %s are: ' %y[flag]
    for i in xrange(0,n):
        print '%s: %s (%s)' %(i+1, largest.index[i], largest[i])
    

hotspots(expr_intervals.loc[(0,0,0,0)], 5, 0)
hotspots(expr_intervals.loc[7], 5, 1)
hotspots(expr_intervals, 5, 2)

Hotspot Analysis Summary
The code regions with largest exclusive time are: 
1: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (2293036.0)
2: void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (2293036.0)
3: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (2108681.0)
4: void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (2108681.0)
5: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **,